In [ ]:
from magnet.ize import memory
from magnet.ron import llm
from magnet.ic import field

field = field.Charge("my-user:T0pS3cr3t@192.168.2.69") # your NATS cluster hostname & basic auth
await field.on(stream="generations", category="nlp_chunk_bge_large_en", create=True)

config = {
    "MILVUS_URI": "192.168.2.69"
    , "DIMENSION": 1024
    , "MODEL": "BAAI/bge-large-en-v1.5"
    , "INDEX": "bge_large_deduped"
    , "INDEX_PARAMS": {
        'metric_type': 'COSINE',
        'index_type':'HNSW',
        'params': {
            "efConstruction": 40
            , "ef": 32768
            , "M": 48
        },
    }
}

gen = llm.Generate(
    hf_token='hf_'
    , field=field
)
async def handle_search(q, docs):
    [print(doc['document']) for doc in docs]
    [print(doc['text']) for doc in docs]
    [print(doc['distance']) for doc in docs]
    return await gen.ask(
        q = q
        , docs = docs
    )

index = memory.Embedder(config)
q = 'Tell me about various AI initiatives'

In [ ]:
from magnet.utils.markdown import m_print

print(f"searching from {index.info().num_entities} chunks")
results = index.search(q, limit=10, cb=handle_search)
results = await results
m_print(results)
index.disconnect()

In [ ]:
await charge.off()